### Titanic survivor prediction

Trying to achieve a good score at the kaggle competion

Points to consider:
- **What to do with the missing values**
    - 1. fill with some values
        - mean, meadian,mode, some conditional values based on other features
    - 2. drop the feature with missing value
    - 3. make a new feature out of the existing feature having missing value
- **Make new features out of existing features**
    - 1. SibSp and Parch
    - 2. Cabin - isna
    - 3. Title from Name
- **Whether to bin some of the features**
    - 1. Age
    - 2. Fare
- **Use minmax scaler/ standard scalar**
- **Use feature selection**
- **Which models to use for prediction**
- **Hypertune**
- **Stacking the models**


### In this notebook we have 3 testcases for 3 models building on previous resutls(tc_7 to tc_9)

- filling age with mean of the group of Pclass,Sex and Title
- **making bins of age**
- dropping cabin
- extracting title from name
- extracting the first letter from ticket
- filling embarked with mode 
- 3 models used RF, LR, XGBoost


In [1]:
import pandas as pd
import numpy as np

In [2]:
# importing data from csv files
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [3]:
df_train.head(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [4]:
# combining the test and train to fill the missing values
comb = df_train.append(df_test)
comb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


## Name

Extracting the titles from the name feature
and dropping the name field



In [5]:
comb['Title'] = comb.Name.str.extract(pat = ' ([A-Za-z]+)\.',expand = False)
''' Expand :If True, return DataFrame with one column per capture group. 
            If False, return a Series/Index if there is one capture group or DataFrame 
            if there are multiple capture groups.
'''
type(comb['Title'])

pandas.core.series.Series

In [6]:
# dropping the name feature
comb.drop('Name',inplace = True,axis = 1)
comb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Sex          1309 non-null   object 
 4   Age          1046 non-null   float64
 5   SibSp        1309 non-null   int64  
 6   Parch        1309 non-null   int64  
 7   Ticket       1309 non-null   object 
 8   Fare         1308 non-null   float64
 9   Cabin        295 non-null    object 
 10  Embarked     1307 non-null   object 
 11  Title        1309 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


## Age

Filling the missing values with median age

In [7]:
comb.Age.isnull().sum()

263

In [8]:
age_median = comb.groupby(['Pclass','Title','Sex'])['Age'].median().reset_index(['Pclass','Title','Sex'])
age_median

,Pclass,Title,Sex,Age
0,1,Capt,male,70.0
1,1,Col,male,54.5
2,1,Countess,female,33.0
3,1,Don,male,40.0
4,1,Dona,female,39.0
5,1,Dr,female,49.0
6,1,Dr,male,47.0
7,1,Jonkheer,male,38.0
8,1,Lady,female,48.0
9,1,Major,male,48.5


In [9]:
age_median[(age_median.Title == 'Ms') & (age_median.Pclass == 2)]['Age'].values[0]

28.0

In [10]:
def get_age(x):
    return age_median[(age_median.Title == x.Title) & \
                      (age_median.Pclass == x.Pclass) & \
                      (age_median.Sex == x.Sex)]['Age'].values[0]


In [11]:
comb.Age = comb.apply(lambda row: get_age(row) if np.isnan(row['Age']) else row['Age'] , axis = 1)
comb.Age.isnull().sum()

1

In [12]:
comb.Age.isnull().sum()

1

In [13]:
comb[comb.Age.isnull()]


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
88,980,NaN,3,female,NaN,0,0,364856,7.75,NaN,Q,Ms


In [14]:
## we fill this value witb median of Pclass 2,Ms as 28
comb.Age.fillna(28,inplace =  True)
comb.Age.isnull().sum()

0

In [15]:
# binning the age
bins= [0,2,4,13,20,75,110]
labels = ['Infant','Toddler','Kid','Teen','Adult','Elderly']
comb['AgeGroup'] = pd.cut(comb['Age'], bins=bins, labels=labels, right=False)


In [16]:
comb.drop('Age',inplace = True, axis = 1)

## Ticket
Extracting the first letter of the ticket


In [17]:
comb.Ticket

0               A/5 21171
1                PC 17599
2        STON/O2. 3101282
3                  113803
4                  373450
              ...        
413             A.5. 3236
414              PC 17758
415    SOTON/O.Q. 3101262
416                359309
417                  2668
Name: Ticket, Length: 1309, dtype: object

In [18]:
# observing the pattern
comb.groupby('Ticket').count()

,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Fare,Cabin,Embarked,Title,AgeGroup
Ticket,,,,,,,,,,,
110152,3,3,3,3,3,3,3,3,3,3,3
110413,3,3,3,3,3,3,3,3,3,3,3
110465,2,2,2,2,2,2,2,2,2,2,2
110469,1,0,1,1,1,1,1,1,1,1,1
110489,1,0,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
W./C. 6608,5,4,5,5,5,5,5,0,5,5,5
W./C. 6609,1,1,1,1,1,1,1,0,1,1,1
W.E.P. 5734,2,1,2,2,2,2,2,2,2,2,2


In [19]:
'''
We observe that some ticket numbers are digits while ohters have an alphabet in the begining.
We use this feature, to keep the first letter of the ticket and insert a 'Z' for the digits only

'''
comb.Ticket = comb.Ticket.map(lambda x: 'Z' if x.isdigit() else x[0] )

In [20]:
comb.groupby('Ticket')['PassengerId'].count()

Ticket
A     42
C     77
F     13
L      5
P     98
S     98
W     19
Z    957
Name: PassengerId, dtype: int64

## Fare
Filling the missing value with mode


In [21]:
comb.Fare.isnull().sum()

1

In [22]:
# filling the. Fare with mode value
comb.Fare.fillna(comb.Fare.mode()[0], inplace = True)
comb.Fare.isnull().sum()

0

## Cabin 
Dropping the feature as it has many missing values


In [23]:
comb.drop(['Cabin'],axis = 1,inplace = True)
comb.head(3)

,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Ticket,Fare,Embarked,Title,AgeGroup
0,1,0.0,3,male,1,0,A,7.2500,S,Mr,Adult
1,2,1.0,1,female,1,0,P,71.2833,C,Mrs,Adult
2,3,1.0,3,female,0,0,S,7.9250,S,Miss,Adult


## Embarked
Filling the missing value with mode

In [24]:
comb.Embarked.isnull().sum()

2

In [25]:

# we first fill embarked with mode value
comb.Embarked.fillna(comb.Embarked.mode()[0], inplace = True)
comb.Embarked.isnull().sum()

0

In [26]:
comb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  1309 non-null   int64   
 1   Survived     891 non-null    float64 
 2   Pclass       1309 non-null   int64   
 3   Sex          1309 non-null   object  
 4   SibSp        1309 non-null   int64   
 5   Parch        1309 non-null   int64   
 6   Ticket       1309 non-null   object  
 7   Fare         1309 non-null   float64 
 8   Embarked     1309 non-null   object  
 9   Title        1309 non-null   object  
 10  AgeGroup     1309 non-null   category
dtypes: category(1), float64(2), int64(4), object(4)
memory usage: 114.0+ KB


## One hot encoding the the categorical features


In [27]:
categorical_feature = comb.select_dtypes(include=['object','category']).columns.tolist()
categorical_feature

['Sex', 'Ticket', 'Embarked', 'Title', 'AgeGroup']

In [28]:
# we include Pclass and parch and sibsp too in the one hot encodeing list
categorical_feature += ['Pclass','Parch','SibSp']
categorical_feature

['Sex', 'Ticket', 'Embarked', 'Title', 'AgeGroup', 'Pclass', 'Parch', 'SibSp']

In [29]:
def onehotencoding(df,colname,prefix):
    dummies = pd.get_dummies(df[colname],prefix = prefix)
    df = pd.concat([df,dummies],axis = 1)
    df.drop(colname,inplace = True, axis = 1)
    return df
    

In [30]:
for feature in categorical_feature:
    comb = onehotencoding(comb,feature,feature)
comb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 58 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PassengerId       1309 non-null   int64  
 1   Survived          891 non-null    float64
 2   Fare              1309 non-null   float64
 3   Sex_female        1309 non-null   uint8  
 4   Sex_male          1309 non-null   uint8  
 5   Ticket_A          1309 non-null   uint8  
 6   Ticket_C          1309 non-null   uint8  
 7   Ticket_F          1309 non-null   uint8  
 8   Ticket_L          1309 non-null   uint8  
 9   Ticket_P          1309 non-null   uint8  
 10  Ticket_S          1309 non-null   uint8  
 11  Ticket_W          1309 non-null   uint8  
 12  Ticket_Z          1309 non-null   uint8  
 13  Embarked_C        1309 non-null   uint8  
 14  Embarked_Q        1309 non-null   uint8  
 15  Embarked_S        1309 non-null   uint8  
 16  Title_Capt        1309 non-null   uint8  
 

In [31]:
comb


,PassengerId,Survived,Fare,Sex_female,Sex_male,Ticket_A,Ticket_C,Ticket_F,Ticket_L,Ticket_P,...,Parch_5,Parch_6,Parch_9,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8
0,1,0.0,7.2500,0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,2,1.0,71.2833,1,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2,3,1.0,7.9250,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,4,1.0,53.1000,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,5,0.0,8.0500,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,8.0500,0,1,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
414,1306,NaN,108.9000,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
415,1307,NaN,7.2500,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
416,1308,NaN,8.0500,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


## prediction 



In [32]:
x_train = comb[0:891]
x_test  = comb[891:]
y_train = x_train['Survived']
x_train = x_train.drop(['Survived','PassengerId'], axis = 1).copy()
x_test  = x_test.drop(['Survived','PassengerId'], axis = 1).copy()
print(x_train.shape,x_test.shape,y_train.shape)

(891, 56) (418, 56) (891,)


In [33]:
x_train

,Fare,Sex_female,Sex_male,Ticket_A,Ticket_C,Ticket_F,Ticket_L,Ticket_P,Ticket_S,Ticket_W,...,Parch_5,Parch_6,Parch_9,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8
0,7.2500,0,1,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,71.2833,1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,7.9250,1,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
3,53.1000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,8.0500,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,13.0000,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
887,30.0000,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
888,23.4500,1,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
889,30.0000,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


## Random Forest

In [34]:
# RF
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=1)
model.fit(x_train,y_train)
result = model.predict(x_test)
result.mean()

0.3660287081339713

In [35]:
result[0:5]

array([0., 0., 0., 0., 0.])

In [36]:
# RF submission file
submission = pd.DataFrame({'PassengerId': df_test.PassengerId,'Survived':result})
submission.Survived = submission.Survived.astype(int)
filename = 'Titanic_predictions_tc_10.csv'
submission.to_csv(filename, index = False)

In [37]:
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


### Score RF: 

## LinearRegression


In [38]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=1)
model.fit(x_train,y_train)
result = model.predict(x_test)
result.mean()

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.38995215311004783

In [39]:
# LR file for submission
submission = pd.DataFrame({'PassengerId': df_test.PassengerId,'Survived':result})
submission.Survived = submission.Survived.astype(int)
filename = 'Titanic_predictions_tc_11.csv'
submission.to_csv(filename, index = False)

### Score LR : 0.78229
## XGBoost

In [40]:
# XGBoost
from xgboost import XGBClassifier
model = XGBClassifier(random_state=1)
model.fit(x_train,y_train)
result = model.predict(x_test)
result.mean()

0.3588516746411483

In [41]:
# XGBoost file for submission
submission = pd.DataFrame({'PassengerId': df_test.PassengerId,'Survived':result})
submission.Survived = submission.Survived.astype(int)
filename = 'Titanic_predictions_tc_12.csv'
submission.to_csv(filename, index = False)

### Score XGBoost: 